PCA 클래스 

In [2]:
#과일 데이터를 다운, 넘파이 배열로 적재 
!wget https://bit.ly/fruits_300_data -O fruits_300.npy

import numpy as np
fruits= np.load('fruits_300.npy')
fruits_2d= fruits.reshape(-1, 100*100)

--2022-08-11 17:30:05--  https://bit.ly/fruits_300_data
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy [following]
--2022-08-11 17:30:05--  https://github.com/rickiepark/hg-mldl/raw/master/fruits_300.npy
Resolving github.com (github.com)... 140.82.113.3
Connecting to github.com (github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy [following]
--2022-08-11 17:30:05--  https://raw.githubusercontent.com/rickiepark/hg-mldl/master/fruits_300.npy
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... conne

In [4]:
#PCA 클래스로 주성분 분석 알고리즘을 제공 

from sklearn.decomposition import PCA
#n_componets 매개변수에 주성분 개수를 지정 
pca = PCA(n_components=50)
#비지도 학습이기에 fit()메서드에 타깃값을 제공하지 않는다 
pca.fit(fruits_2d)

PCA(n_components=50)

In [5]:
#PCA 클래스가 찾은 주성분은 compoents_속성에 저장
#배열의 크기를 확인하기
print(pca.components_.shape)

(50, 10000)
